Inference\
https://wikidocs.net/115055

https://huggingface.co/docs/transformers/index

http://kkma.snu.ac.kr/documents/index.jsp?doc=postag


# Author: Yoonhyuck WOO / JBNU_Industrial Information system Engineering
# Date; 2. 10. 2022 - 2. 22. 2022
# Title: Korean_NER
# Professor: Seung-Hoon Na

In [1]:
import os
import pandas as pd
import json
import transformers
from transformers import BertTokenizer
# from tqdm.auto import tqdm

In [2]:
PATH_dir = 'C:\\Users\\LG\\Desktop\\github\\JBNU-2022-SPRING\\English world class tagging & Korean_Named Entity Recognition\\Ko_En_NER_POStag_data\Ko_NER_POS'
file_name_ko_train = 'train.txt'
file_name_ko_test = 'test.txt'
file_name_ko_dev = 'dev.txt'

# temp

In [3]:
class Data_prepro:
    
    def make_json(file_name):
        PATH_data = os.path.join(PATH_dir, file_name)
        prepro_file_name = 'prepro_' + file_name.rsplit('.')[0] + '.json' #change file extention from '.txt' to '.json'
        PATH_preprodata = os.path.join(PATH_dir, prepro_file_name)
        
    def read_file(path):
        with open(PATH_data, 'r', encoding='UTF8') as f:
            rawdata = f.readlines()
            
        return rawdata
        
    def Data_NER(path):
        
        # data preprocessing by using pandas
        df0 = pd.DataFrame(rawdata)
        df1 = pd.DataFrame(df0[0].str.split('/').tolist(),columns=['1-1','1-2'])
        df2 = pd.DataFrame(df0[0].str.split('\t').tolist(),columns=['1','2','3','4'])
        df_final = pd.concat([df1['1-1'],df2['4'].str.strip('\n')],axis = 1)
        df_final = pd.DataFrame(df_final)
        df_final = df_final.rename(columns={'1-1':'Entity','4':'tag'})
        
        PRETAINED_MODEL_NAME = 'bert-base-multilingual-cased'
        tokenizer = BertTokenizer.from_pretrained(PRETAINED_MODEL_NAME)
        
        # make dataset & tokenizer_tagging [original, token, tokenizer_tagging]
        lst=[]
        lst_set = []
        lst_ent = []
        lst_tag = []
        for i in range(6945):
            if df_final['tag'][i] != None:
                morph_to_tokens = tokenizer.tokenize(df_final['Entity'][i])
                if len(df_final['Entity'][i]) == len(morph_to_tokens[0]):
                    lst_ent.append(df_final['Entity'][i])
                    lst_tag.append(df_final['tag'][i])

                else:
                    if df_final['tag'][i] == 'O':
                        lst_ent.append(df_final['Entity'][i])
                        lst_tag.append(df_final['tag'][i])
                        for i in range(len(morph_to_tokens)-1):
                            lst_tag.append(df_final['tag'][i])

                    else:
                        lst_ent.append(df_final['Entity'][i])
                        lst_tag.append(df_final['tag'][i])
                        split = df_final['tag'][i].split('-')
                        for i in range(len(morph_to_tokens)-1):
                            results = 'I-' + split[-1]
                            lst_tag.append(results)

            else:
                lst_ent = result = ' '.join(str(s) for s in lst_ent)
                lst_set.append(lst_ent)
                morph_to_tokens = tokenizer.tokenize(lst_ent)
                lst_set.append(morph_to_tokens)
                lst_set.append(lst_tag)
                lst.append(lst_set)
                result = 0
                lst_ent = []
                lst_tok = []
                lst_tag = []
                lst_set = []
            
        return lst
    
    
    def rewrite_json():
        with open(PATH_preprodata, 'w') as f:
            json.dump(lst, f)
            
        with open(PATH_preprodata, 'r') as f:
            preprodata = json.load(f)    

# ======================================

In [11]:
def make_json(file_name):
    global df_final
    PATH_data = os.path.join(PATH_dir, file_name)

    #change file extention from '.txt' to '.json'
    prepro_file_name = 'prepro_' + file_name.rsplit('.')[0] + '.json'
    PATH_preprodata = os.path.join(PATH_dir, prepro_file_name)

    with open(PATH_data, 'r', encoding='UTF8') as f:
        rawdata = f.readlines()

    # data preprocessing by using pandas
    df0 = pd.DataFrame(rawdata)
    if file_name == 'dev.txt':
        df1 = pd.DataFrame(df0[0].str.split('/').tolist(),columns=['1-1','1-2'])
        df2 = pd.DataFrame(df0[0].str.split('\t').tolist(),columns=['1','2','3','4'])
        df_final = pd.concat([df1['1-1'],df2['4'].str.strip('\n')],axis = 1)
        df_final = pd.DataFrame(df_final)
        df_final = df_final.rename(columns={'1-1':'Entity','4':'tag'})
    else:
        df1 = pd.DataFrame(df0[0].str.split('/').tolist(),columns=['1-1','1-2','1-3'])
        df2 = pd.DataFrame(df0[0].str.split('\t').tolist(),columns=['1','2','3','4'])
        df_final = pd.concat([df1['1-1'],df2['4'].str.strip('\n')],axis = 1)
        df_final = pd.DataFrame(df_final)
        df_final = df_final.rename(columns={'1-1':'Entity','4':'tag'})
   
    # upload tokenizer
    PRETAINED_MODEL_NAME = 'bert-base-multilingual-cased'
    tokenizer = BertTokenizer.from_pretrained(PRETAINED_MODEL_NAME)

    # make dataset & tokenizer_tagging [original, token, tokenizer_tagging]
    lst=[]
    lst_set = []
    lst_ent = []
    lst_tag = []
    for i in range(len(df_final)):
        if df_final['tag'][i] != None:
            morph_to_tokens = tokenizer.tokenize(df_final['Entity'][i])
            if 1 == len(morph_to_tokens):
                lst_ent.append(df_final['Entity'][i])
                lst_tag.append(df_final['tag'][i])

            else:
                if df_final['tag'][i] == 'O':
                    lst_ent.append(df_final['Entity'][i])
                    lst_tag.append(df_final['tag'][i])
                    for i in range(len(morph_to_tokens)-1):
                        lst_tag.append(df_final['tag'][i])

                else:
                    lst_ent.append(df_final['Entity'][i])
                    lst_tag.append(df_final['tag'][i])
                    split = df_final['tag'][i].split('-')
                    for i in range(len(morph_to_tokens)-1):
                        results = 'I-' + split[-1]
                        lst_tag.append(results)

        else:
            lst_ent = result = ' '.join(str(s) for s in lst_ent)
            lst_set.append(lst_ent)
            morph_to_tokens = tokenizer.tokenize(lst_ent)
            lst_set.append(morph_to_tokens)
            lst_set.append(lst_tag)
            lst.append(lst_set)
            result = 0
            lst_ent = []
            lst_tok = []
            lst_tag = []
            lst_set = []

    with open(PATH_preprodata, 'w') as f:
        json.dump(lst, f)

    with open(PATH_preprodata, 'r') as f:
        preprodata = json.load(f)

In [7]:
prepro = Data_prepro()

train_file = prepro.make_json(file_name_ko_train)
test_file = prepro.make_json(file_name_ko_test)
dev_file = prepro.make_json(file_name_ko_dev)



In [ ]:
file_name_ko_train = 'train.txt'
file_name_ko_test = 'test.txt'
#file_name_ko_dev = 'dev.txt'

In [12]:
make_json(file_name_ko_train)
make_json(file_name_ko_test)
make_json(file_name_ko_dev)

In [ ]:
make_json(file_name_ko_test)

In [5]:
make_json(file_name_ko_dev)

UnboundLocalError: local variable 'df_final' referenced before assignment

# My Data

In [95]:
result = ' '.join(str(s) for s in lst[3][0])

In [96]:
inputs = tokenizer(result, return_tensors='pt')
print(inputs['input_ids'].size())
inputs

torch.Size([1, 85])


{'input_ids': tensor([[   101,  10386,   9367,   9246,  10739, 119327,   9792,  73352,   9637,
           9783,  75855,   9909,  12605,   9233,   9580,  37819,  22028,   9776,
          14867,  24178,  64749, 118965,   9632,   9319,  12508,  17138,   9632,
           9666, 118943,   9954,   8857,  12030,  12310,   9202,   9309,  27023,
          24891,  52015,  30873,   9460,  29455,  15891,   9233,   9672,  55358,
           9109,   8892,  25934,  29805,   8853,  35506,   8888,   9670, 119445,
           9954,   9909,  12605,   9233,   9010,   9557,   8888,   9213,  25503,
           8843,   9638,   9233,   9580,  37819,  51431,   9476,  29805,   9568,
          74322,    117,   9484,  14646,   9233,   9612,  34907,  29805,   9091,
         118881,   9056,    119,    102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 

In [97]:
from transformers import BertModel

In [98]:
PRETAINED_MODEL_NAME = 'bert-base-multilingual-cased'
bert = BertModel.from_pretrained(PRETAINED_MODEL_NAME)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [99]:
outputs = bert(**inputs)

In [100]:
outputs.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [101]:
outputs

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.1460, -0.3585, -0.1289,  ..., -0.0559, -0.0949, -0.0022],
         [ 0.4797, -0.2805, -0.3411,  ...,  0.5483, -0.2645,  0.1313],
         [ 0.2723, -0.2698,  0.2306,  ...,  0.1069,  0.2373, -0.5147],
         ...,
         [-0.4482, -0.5314, -0.1884,  ..., -0.4645,  0.2049, -0.6246],
         [-0.1095, -0.4616, -0.7723,  ..., -0.0311,  0.1242, -0.2750],
         [ 0.2146, -0.3461, -0.3170,  ..., -0.1243,  0.1938, -0.1037]]],
       grad_fn=<NativeLayerNormBackward>), pooler_output=tensor([[ 1.4693e-01, -2.6310e-01,  2.1919e-01, -3.1868e-01, -3.4607e-01,
          3.1139e-02,  1.9270e-01, -3.6483e-02, -1.0906e-01, -1.0840e-01,
         -1.2835e-01,  1.8402e-02, -8.5716e-02,  3.7373e-02,  1.9826e-01,
          1.7237e-01, -1.5535e-01,  1.9489e-01,  1.3541e-01,  6.7832e-02,
         -9.8824e-01, -2.3051e-01,  1.3823e-01, -1.9326e-01,  3.8057e-02,
          2.1562e-01, -3.1980e-01,  1.1865e-01, -7.8593e-03, -3.1718